### Data prep

- Downloaded CSV of travis dataset 
- use first 100 commits of dataset 

- extract commits id & project name of those commits
- use values to get corresponding pom files (previous and current)
- use differ to extract information from pom files
- use this as data and the build result of the 100 travis commits as labels

In [45]:
#!pip3 install pandas
import pandas as pd 

# TODO just use if original travistorrent dataset is present instead of smaller version 
# size down original file
df = pd.read_csv('/Users/Rina/Desktop/StudiAssistent/Projekt_Bewerbung/data/travistorrent_5_3_2016.csv')

# TODO get all four labels as previous tests showed that most labels are errored and we only get 1 label with picking 100commits naivly (even with shuffling)

# errored, failed, passed, canceled
df_failed = df[(df['tr_status']=='failed') & (df['tr_analyzer'] == 'java-maven') & (df['gh_lang'] == 'java')][:25] # others don't have pom.xml
df_passed = df[(df['tr_status']=='passed') & (df['tr_analyzer'] == 'java-maven') & (df['gh_lang'] == 'java')][:25]
df_canceled = df[(df['tr_status']=='canceled') & (df['tr_analyzer'] == 'java-maven') & (df['gh_lang'] == 'java')] [:25]
df_errored = df[(df['tr_status']=='errored') & (df['tr_analyzer'] == 'java-maven') & (df['gh_lang'] == 'java')] [:25] 

vertical_stack = pd.concat([df_canceled, df_errored, df_failed, df_passed], axis=0)
print(vertical_stack.shape)
vertical_stack = vertical_stack.sample(frac = 1) # shuffle dataset (1=100%)

vertical_stack.to_csv('/Users/Rina/Desktop/StudiAssistent/Projekt_Bewerbung/data/travistorrent_100commits_all4classes_shuffled.csv', sep=',')
vertical_stack.head()

/var/folders/fq/98klxnfs5snds3nsbbmn85nw0000gn/T/ipykernel_1910/4078726169.py:6: DtypeWarning: Columns (4,42,47,48,49,50,51,52,53,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/Rina/Desktop/StudiAssistent/Projekt_Bewerbung/data/travistorrent_5_3_2016.csv')


(100, 56)


,row,git_commit,gh_project_name,gh_is_pr,git_merged_with,gh_lang,git_branch,gh_first_commit_created_at,gh_team_size,git_commits,...,tr_tests_skipped,tr_failed_tests,tr_testduration,tr_purebuildduration,tr_tests_ran,tr_tests_failed,git_num_committers,tr_num_jobs,tr_prev_build,tr_ci_latency
479,480,6ddd576e8fffb97e7b24df85fd11ae2191c8e45e,ActiveJpa/activejpa,False,NaN,java,master,2013-12-06 03:57:52,2,dc12f25c7126ab7f8edfbe6827cf4a2f6f0652c7,...,0.0,NaN,19.62,0.0,True,False,1,3,203277.0,74
34847,34848,bfd12bb506dea9b9a615fb4886d9bb98d8dca8b0,Graylog2/graylog2-server,False,NaN,java,master,2015-05-20 18:08:49,7,68d88ca3eb05cce8124f6b7af2bd272ce49d61aa#32b3e...,...,0.0,NaN,771.58,93.0,True,False,2,2,-1.0,14828
34845,34846,bfd12bb506dea9b9a615fb4886d9bb98d8dca8b0,Graylog2/graylog2-server,False,NaN,java,master,2015-05-20 18:08:49,7,68d88ca3eb05cce8124f6b7af2bd272ce49d61aa#32b3e...,...,0.0,NaN,993.31,146.0,True,False,2,2,-1.0,14828
481,482,6ddd576e8fffb97e7b24df85fd11ae2191c8e45e,ActiveJpa/activejpa,False,NaN,java,master,2013-12-06 03:57:52,2,dc12f25c7126ab7f8edfbe6827cf4a2f6f0652c7,...,0.0,NaN,24.97,0.0,True,False,1,3,192057.0,74
121157,121158,2f0ebdafec8a47bea4a37ba9e5782df3b0ea315c,SonarSource/sonarqube,False,NaN,java,master,2015-08-10 08:25:20,18,c644a4df6cb5d86ca51374e14010e883cb22ca5b#63dd3...,...,23.0,NaN,36.02,67.0,True,False,2,11,-1,6


In [46]:
# read in file 
df = pd.read_csv('/Users/Rina/Desktop/StudiAssistent/Projekt_Bewerbung/data/travistorrent_100commits_all4classes_shuffled.csv')
print('Shape of dataframe', df.shape)

Shape of dataframe (100, 57)


In [47]:
df.head()

,Unnamed: 0,row,git_commit,gh_project_name,gh_is_pr,git_merged_with,gh_lang,git_branch,gh_first_commit_created_at,gh_team_size,...,tr_tests_skipped,tr_failed_tests,tr_testduration,tr_purebuildduration,tr_tests_ran,tr_tests_failed,git_num_committers,tr_num_jobs,tr_prev_build,tr_ci_latency
0,479,480,6ddd576e8fffb97e7b24df85fd11ae2191c8e45e,ActiveJpa/activejpa,False,NaN,java,master,2013-12-06 03:57:52,2,...,0.0,NaN,19.62,0.0,True,False,1,3,203277.0,74
1,34847,34848,bfd12bb506dea9b9a615fb4886d9bb98d8dca8b0,Graylog2/graylog2-server,False,NaN,java,master,2015-05-20 18:08:49,7,...,0.0,NaN,771.58,93.0,True,False,2,2,-1.0,14828
2,34845,34846,bfd12bb506dea9b9a615fb4886d9bb98d8dca8b0,Graylog2/graylog2-server,False,NaN,java,master,2015-05-20 18:08:49,7,...,0.0,NaN,993.31,146.0,True,False,2,2,-1.0,14828
3,481,482,6ddd576e8fffb97e7b24df85fd11ae2191c8e45e,ActiveJpa/activejpa,False,NaN,java,master,2013-12-06 03:57:52,2,...,0.0,NaN,24.97,0.0,True,False,1,3,192057.0,74
4,121157,121158,2f0ebdafec8a47bea4a37ba9e5782df3b0ea315c,SonarSource/sonarqube,False,NaN,java,master,2015-08-10 08:25:20,18,...,23.0,NaN,36.02,67.0,True,False,2,11,-1.0,6


#### Get DataFrame with information on project and commit of which we have build results

In [48]:
commit_information = pd.DataFrame(df)
commit_information = commit_information[['gh_project_name', 'git_commit', 'tr_status']]#.drop_duplicates()[0:100]
print(commit_information) # now we have a dataframe containing 
print('Size of dataframe (rows): ', commit_information.shape[0])

             gh_project_name                                git_commit  \
0        ActiveJpa/activejpa  6ddd576e8fffb97e7b24df85fd11ae2191c8e45e   
1   Graylog2/graylog2-server  bfd12bb506dea9b9a615fb4886d9bb98d8dca8b0   
2   Graylog2/graylog2-server  bfd12bb506dea9b9a615fb4886d9bb98d8dca8b0   
3        ActiveJpa/activejpa  6ddd576e8fffb97e7b24df85fd11ae2191c8e45e   
4      SonarSource/sonarqube  2f0ebdafec8a47bea4a37ba9e5782df3b0ea315c   
..                       ...                                       ...   
95       ActiveJpa/activejpa  6ddd576e8fffb97e7b24df85fd11ae2191c8e45e   
96       ActiveJpa/activejpa  1432c0bb15162a8b15b73d908f43d6f2e5b54c90   
97     dropwizard/dropwizard  de86264f94e67513c2f638b29e5326678225613a   
98             DSpace/DSpace  6b116c496d6e474459e5dc9aea2799845d007c9d   
99       ActiveJpa/activejpa  ad482f5c88113ebc484bce577ca207a393ba648b   

   tr_status  
0     passed  
1   canceled  
2   canceled  
3     passed  
4   canceled  
..       ...  
95    

#### Clone project, checkout commit id and download current pom and pom before the commit 

In [49]:
# Imports
#!pip3 install GitPython
import git 
import shutil
import os

In [50]:
# path to pom file of a project using maven 
prefix_project = '/Users/Rina/Desktop/StudiAssistent/Projekt_Bewerbung/'
path_to_pom = prefix_project+'dataprep/projects/pom.xml'
prefix_url = 'https://github.com/'

# method for getting Pom file from commit id 
import shutil
def getPomFile(project, commit_id, path_to_pom, path_to_save_to):
    project.git.checkout(commit_id) # checkout commit of data to receive pom.xml file
    shutil.copy(path_to_pom, path_to_save_to)
    #!cp path_to_pom path_to_save_to

In [ ]:
# TODO addd error handling as some commits not present anymore 

In [54]:
for index, row in commit_information.iterrows():
    # do it for every commit 
    commit_id = row['git_commit']
    project_name = row['gh_project_name']

    project_url = prefix_url+project_name
    if os.path.exists('projects') and os.path.isdir('projects'):
        shutil.rmtree('projects')
    project = git.Repo.clone_from(project_url, '/Users/Rina/Desktop/StudiAssistent/Projekt_Bewerbung/dataprep/projects')

    # TODO adapt naming of subfolders (commit id's are present multiple times so i stayed with indices for now)
    base_path = prefix_project+'data/pom_files/'+str(index)#+'/'+project_name+'/'+commit_id
    os.makedirs(base_path)

    path_to_save_to_current = base_path+'/pom_current.xml'
    path_to_save_to_previous = base_path+'/pom_previous.xml'

    getPomFile(project, commit_id, path_to_pom, path_to_save_to_current)
    prev_commit_id = commit_id+'^1'
    project.git.checkout(prev_commit_id) # receive pom.xml file before commit was done
    getPomFile(project, prev_commit_id, path_to_pom, path_to_save_to_previous)
    with open(base_path+"/build_result.csv", "w") as file:
        file.write(row['tr_status'])

    # TODO run for all 100 commits to get all pom files into working directory for differ :) 
    #if index == 1:
    #    break
    print(index, '; commit:', commit_id)


0 ; commit: 6ddd576e8fffb97e7b24df85fd11ae2191c8e45e
1 ; commit: bfd12bb506dea9b9a615fb4886d9bb98d8dca8b0
2 ; commit: bfd12bb506dea9b9a615fb4886d9bb98d8dca8b0
3 ; commit: 6ddd576e8fffb97e7b24df85fd11ae2191c8e45e
4 ; commit: 2f0ebdafec8a47bea4a37ba9e5782df3b0ea315c
5 ; commit: a78844b0b83458912efa76aeff5631ffaac9cdcb
6 ; commit: de86264f94e67513c2f638b29e5326678225613a
7 ; commit: 2f0ebdafec8a47bea4a37ba9e5782df3b0ea315c
8 ; commit: 747a1c4f629ffac24552535501b9b4d48bbb92bb
9 ; commit: 9b1b5e19d483a1417f7cdbabd3587215cfe9d9a6
10 ; commit: 698ba1bd0283fc1ce1c12dd184987960e3025013
11 ; commit: bfd12bb506dea9b9a615fb4886d9bb98d8dca8b0
12 ; commit: dc12f25c7126ab7f8edfbe6827cf4a2f6f0652c7
13 ; commit: 1432c0bb15162a8b15b73d908f43d6f2e5b54c90
14 ; commit: 3ec9b40baf57b647de428aed5a8e0571cd9d5d8a
15 ; commit: 78afe1925d8d02a28516cc3768e8fc42e3d7b71d
16 ; commit: 9b1b5e19d483a1417f7cdbabd3587215cfe9d9a6
17 ; commit: 749da4f3a8524fc0e9e72317cf2ccc6eeafa0086
18 ; commit: 52f7145d808cb14761a6285e9

GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git checkout 8ff99d8a218815fe4b9181a4f84df2427ab9fe47
  stderr: 'fatal: reference is not a tree: 8ff99d8a218815fe4b9181a4f84df2427ab9fe47'